In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import re

import geopandas as gpd
import requests,json


from shapely import geos
from shapely.geometry import Point
import fiona
from fiona.crs import from_epsg,from_string

from geopandas import GeoDataFrame
from shapely.geometry import Polygon
from geopandas import GeoSeries


from shapely import geometry
from shapely.geometry import shape,LineString,Point,Polygon

import osmnx as ox
import networkx as nx

import folium

from folium import IFrame

import seaborn as sns
import branca

In [3]:
def rest(address):
    try:
        url = 'https://api.tomtom.com/search/2/geocode/{}.json?&key=h2n93GznJsnsDHtv2vPpdT9GmJijp9o9&countrySet=TWN&language=zh-TW&limit=1'.format(address)
        response = requests.get(url)
        data = response.text
        js = json.loads(str(data))
        return js['results'][0]['position']
    except Exception as e:
        print(f'An Error occurred: {e}')
        return {'lat':0,'lon':0}
    
def trans(gdf):
    gdf.crs = {'init' :'epsg:3824'} # 避免資料沒設，這邊再重新給一次
    gdf=gdf.to_crs({'init' :'epsg:4326'})
    return gdf

def utf2asc(s):
    return str(str(s).encode('ascii', 'xmlcharrefreplace'))[2:-1]

heading3 = """<h3>{}</h3>""".format

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import os
os.chdir(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data')

# 村里邊界資料整理

In [6]:
gdf=gpd.read_file(r'C:/Users/RT009223/GIS/村里/VILLAGE_MOI_1090423.shp',encoding='utf-8')
gdf=gdf.loc[:,['COUNTYNAME','TOWNNAME','VILLNAME','geometry']]
gdf=trans(gdf)


In [7]:
gdf=gdf.dropna().reset_index(drop=True)

In [8]:
gdf.COUNTYNAME=list(gdf.COUNTYNAME.apply(lambda x:x.replace('臺','台')))

In [9]:
print(gdf.shape)
gdf.columns=['縣市','行政區','村里','geometry']
gdf.head()


(7760, 4)


,縣市,行政區,村里,geometry
0,高雄市,林園區,中門里,"POLYGON ((120.36772 22.49564, 120.36427 22.498..."
1,高雄市,林園區,港埔里,"POLYGON ((120.37325 22.49123, 120.37147 22.492..."
2,高雄市,旗津區,上竹里,"POLYGON ((120.28976 22.57317, 120.28577 22.577..."
3,高雄市,旗津區,中興里,"POLYGON ((120.30320 22.55568, 120.30284 22.557..."
4,連江縣,南竿鄉,津沙村,"POLYGON ((119.91684 26.14109, 119.91683 26.141..."


In [10]:
#gdf[gdf['村里']=='明德里'].area

In [11]:
#gdf[gdf['村里']=='明德里'].to_crs({'init': 'epsg:3857'}).area/ 10**6

In [12]:
#gdf[gdf['村里']=='明德里'].to_crs({'init': 'epsg:3395'}).area/ 10**6

# 會員資料整理

In [13]:
df_get_back=pd.read_table(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\HOLA會員位置(無正確地址的資料)_一般卡_2019.txt',sep=',')

In [14]:
df_get_back=df_get_back[-df_get_back['村里'].isna()].reset_index(drop=True)

In [15]:
df_get_back.head()

,id,縣市,行政區,村里
0,20150928000108,新北市,三峽區,龍埔里
1,20040724000470,桃園市,楊梅區,雙榮里
2,20020328000728,高雄市,苓雅區,和煦里
3,19980324001785,高雄市,鳳山區,中崙里
4,19980731000270,新北市,樹林區,樂山里


# Mapping資料

In [16]:
#geom=[Point(data.lon,data.lat) for idx, data in df_get_back.iterrows()]

In [17]:
#crs = {'init': 'epsg:4326'}
#df_get_back = gpd.GeoDataFrame(df_get_back, crs=crs, geometry=geom)

In [18]:
#df_get_back.shape

In [19]:
#df_get_back = gpd.tools.sjoin(df_get_back,gdf, how="inner", op='intersects')
#df_get_back.head()

In [20]:
#df_get_back.columns=['id','addr','zone','lon','lat','geometry','index_right','縣市','行政區','村里']
#df_get_back=df_get_back.loc[:,['id','addr','lon','lat','縣市','行政區','村里']]
#df_get_back.head()

In [21]:
#result.columns=['id','addr','lon','lat','geometry','index_right','縣市','行政區','村里']
#result=result.copy().reset_index(drop=True)

In [22]:
#result2=result.copy()

In [23]:
#result=result.iloc[:,1:]
#result.head()

In [24]:
#result.to_csv('TLW會員位置(經整理_配對村里)_一般卡.txt',index=0)

In [25]:
result=pd.read_table(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\HOLA會員位置(經整理_配對村里)_一般卡_2019.txt',sep=',')
print(result.shape)
result.head()

(800254, 7)


,id,addr,lon,lat,縣市,行政區,村里
0,20190427000460,新北市新莊區龍鳳里28鄰中正路893巷110號 ...,121.414699,25.020392,新北市,新莊區,龍鳳里
1,20121222000078,新北市新莊區光華里17鄰西盛街405號 ...,121.427707,25.012607,新北市,新莊區,光華里
2,20170810000036,新竹市東區綠水里10鄰光復路二段301號 ...,120.989550,24.799584,新竹市,東區,綠水里
3,20030515003473,新北市樹林區光興里7鄰光興街20號 ...,121.412377,25.002980,新北市,樹林區,光興里
4,20011011000048,新北市中和區平河里1鄰連城路258號 ...,121.487238,24.996445,新北市,中和區,平河里


In [26]:
df_get_back=df_get_back[-df_get_back.id.isin(result.id)]

In [27]:
result=pd.concat([result,df_get_back]).reset_index(drop=True)

In [28]:
result.shape

(812567, 7)

# 會員消費額

In [29]:
sales=pd.read_csv(r'C:\Users\RT009223\TestriteGroup\滲透率專案\01_raw data\HOLA2019一般卡會員消費金額.csv',error_bad_lines=False,header=None)
sales.columns=['id','sale']
print(sales.shape)
print(sales.sale.sum())
print(sales.id.isin(result.id).sum())

(917855, 2)
3701721373.2199993
812563


In [30]:
result=pd.merge(result,sales).reset_index(drop=True)
result.shape

(812563, 8)

# 人口資料整理

In [74]:
pop=pd.read_excel(r'台灣2019年村里人口.xlsx')

In [75]:
pop['縣市']=pop['縣市'].str.replace('\xa0','')
pop['行政區']=pop['行政區'].str.replace('\xa0','')
pop['村里']=pop['村里'].str.replace('\xa0','')

In [76]:

#公舘里~公館里
#上舘里~上館里
#石[曹]里~石𥕢里
#鷄林里~雞林里
#水磜村~水祭村
#珊湖里~珊瑚里

In [77]:
gdf.columns=['縣市','行政區','村里','geometry']

In [78]:
gdf['縣市']=list(gdf['縣市'].apply(lambda x:x.replace('臺','台')))
gdf['行政區']=list(gdf['行政區'].apply(lambda x:x.replace('臺','台')))
gdf['村里']=list(gdf['村里'].apply(lambda x:x.replace('臺','台')))
pop['縣市']=list(pop['縣市'].apply(lambda x:x.replace('臺','台')))
pop['行政區']=list(pop['行政區'].apply(lambda x:x.replace('臺','台')))
pop['村里']=list(pop['村里'].apply(lambda x:x.replace('臺','台')))

gdf['村里']=gdf['村里'].apply(lambda x:x.replace('舘','館'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('舘','館'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('双','雙'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('双','雙'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))

gdf['村里']=gdf['村里'].apply(lambda x:x.replace('欍','瓊'))

pop['村里']=pop['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))

pop['村里']=pop['村里'].apply(lambda x:x.replace('欍','瓊'))


pop['村里']=pop['村里'].apply(lambda x:x.replace('溝垻里','溝埧里'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('崙峰里','崙峯里'))

In [79]:
result['村里']=result['村里'].apply(lambda x:x.replace('舘','館'))
result['村里']=result['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
result['村里']=result['村里'].apply(lambda x:x.replace('双','雙'))
result['村里']=result['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
result['村里']=result['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
result['村里']=result['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))
result['村里']=result['村里'].apply(lambda x:x.replace('欍','瓊'))

In [80]:
#result2=result.groupby(['縣市','行政區','村里'])['id'].count().reset_index()
result2=result.groupby(['縣市','行政區','村里']).agg({'id':'count', 'sale': 'sum'}).reset_index()
result2.columns=['縣市','行政區','村里','會員數','消費額']
result2.shape

(7554, 5)

In [81]:
result2['平均消費額']=round(result2['消費額']/result2['會員數'])

In [83]:
result2=pd.merge(pop,result2,how='left')
result2.shape

(7605, 8)

In [84]:
result2[-(result2['縣市']+result2['行政區']+result2['村里']).isin(pop['縣市']+pop['行政區']+pop['村里'])]['縣市'].unique()

array([], dtype=object)

In [85]:
pop[-(pop['行政區']+pop['村里']).isin(gdf['行政區']+gdf['村里'])]

,縣市,行政區,村里,戶口,人口


In [86]:
#gdf[(gdf['行政區']+gdf['村里']).apply(lambda x:x=='龍崎區崎頂里')]

In [87]:
#pop[pop['行政區']=='龍崎區']['村里'].unique()

In [88]:
#pop[pop['村里']=='崎頂里']

In [89]:
#gdf[gdf['村里']=='崎頂里']

In [90]:
#gdf[gdf['行政區']=='新化區']['村里'].unique()

In [91]:
result2['會員數'].sum()

812172.0

In [92]:
gdf_final=pd.merge(gdf,result2,how='right')

In [93]:
gdf_final['會員數'][gdf_final['會員數'].isna()]=0

In [94]:
gdf_final['site']=gdf_final['縣市']+gdf_final['行政區']+gdf_final['村里']
gdf_final

,縣市,行政區,村里,geometry,戶口,人口,會員數,消費額,平均消費額,site
0,高雄市,林園區,中門里,"POLYGON ((120.36772 22.49564, 120.36427 22.498...",1120,2894,47.0,122902.19,2615.0,高雄市林園區中門里
1,高雄市,林園區,港埔里,"POLYGON ((120.37325 22.49123, 120.37147 22.492...",819,2060,28.0,41922.00,1497.0,高雄市林園區港埔里
2,高雄市,旗津區,上竹里,"POLYGON ((120.28976 22.57317, 120.28577 22.577...",1391,3254,46.0,63190.09,1374.0,高雄市旗津區上竹里
3,高雄市,旗津區,中興里,"POLYGON ((120.30320 22.55568, 120.30284 22.557...",1064,2877,58.0,83960.71,1448.0,高雄市旗津區中興里
4,屏東縣,潮州鎮,新榮里,"POLYGON ((120.54005 22.54951, 120.54019 22.549...",594,1394,22.0,63775.00,2899.0,屏東縣潮州鎮新榮里
...,...,...,...,...,...,...,...,...,...,...
7600,基隆市,七堵區,實踐里,"POLYGON ((121.70123 25.08284, 121.70167 25.081...",1691,4446,83.0,413926.28,4987.0,基隆市七堵區實踐里
7601,基隆市,七堵區,堵北里,"MULTIPOLYGON (((121.69626 25.08189, 121.69423 ...",1604,4252,94.0,563568.17,5995.0,基隆市七堵區堵北里
7602,基隆市,七堵區,堵南里,"POLYGON ((121.69469 25.08739, 121.69373 25.086...",936,2359,57.0,316720.18,5556.0,基隆市七堵區堵南里
7603,基隆市,暖暖區,暖東里,"POLYGON ((121.74707 25.09037, 121.74748 25.090...",188,464,8.0,146011.00,18251.0,基隆市暖暖區暖東里


In [95]:
gdf_final['ratio']=gdf_final['會員數']/gdf_final['戶口']
gdf_final['ratio2']=gdf_final['ratio']
gdf_final.loc[gdf_final['ratio2']>0.4,'ratio2']=0.4
gdf_final['percent']=gdf_final['ratio'].map(lambda x:str(round(x*100,1))+'%')
#gdf_final=gdf_final.set_index(gdf_final.site)
gdf_final['MarketPotential']=(gdf_final['戶口']-gdf_final['會員數']).astype('int')

In [96]:
gdf_final['平均消費額'].describe()

count     7463.000000
mean      3661.207423
std       2176.413815
min         67.000000
25%       2580.500000
50%       3462.000000
75%       4332.000000
max      47764.000000
Name: 平均消費額, dtype: float64

In [97]:
gdf_final['平均消費額2']=gdf_final['平均消費額']
gdf_final['平均消費額2'][gdf_final['平均消費額']>10000]=10000

In [98]:
#plt.boxplot(gdf_final.MarketPotential)

In [99]:
#gdf_final.sort_values(by='ratio',ascending=False).head(10)

In [100]:
#(result.addr.str.contains('保新里')&result.addr.str.contains('汐止區')).sum()

In [101]:
#result[result['村里']=='保新里'].head()

In [102]:
#gdf_final.sort_values(by=['ratio','戶口'],ascending=[True,False]).head(10)

In [103]:
gdf_final['MarketPotential_normalization']=(gdf_final['MarketPotential']-gdf_final['MarketPotential'].min())/(gdf_final['MarketPotential'].max()-gdf_final['MarketPotential'].min())
gdf_final['MarketPotential_density']=(gdf_final['MarketPotential'])/(gdf_final.to_crs({'init': 'epsg:3395'}).area/ 10**6)
gdf_final['MarketPotential_density_normalization']=(gdf_final['MarketPotential_density']-gdf_final['MarketPotential_density'].min())/(gdf_final['MarketPotential_density'].max()-gdf_final['MarketPotential_density'].min())
gdf_final['percent']=gdf_final['ratio'].map(lambda x:str(round(x*100,1))+'%')

In [104]:
gdf_final['平均消費額_normalization']=(gdf_final['平均消費額2']-gdf_final['平均消費額2'].min())/(gdf_final['平均消費額2'].max()-gdf_final['平均消費額2'].min())
gdf_final.loc[gdf_final['平均消費額_normalization'].isna(),['平均消費額_normalization']]=0

In [105]:
gdf_final.head()

,縣市,行政區,村里,geometry,戶口,人口,會員數,消費額,平均消費額,site,ratio,ratio2,percent,MarketPotential,平均消費額2,MarketPotential_normalization,MarketPotential_density,MarketPotential_density_normalization,平均消費額_normalization
0,高雄市,林園區,中門里,"POLYGON ((120.36772 22.49564, 120.36427 22.498...",1120,2894,47.0,122902.19,2615.0,高雄市林園區中門里,0.041964,0.041964,4.2%,1073,2615.0,0.077224,436.252854,0.010665,0.256519
1,高雄市,林園區,港埔里,"POLYGON ((120.37325 22.49123, 120.37147 22.492...",819,2060,28.0,41922.00,1497.0,高雄市林園區港埔里,0.034188,0.034188,3.4%,791,1497.0,0.056405,517.365797,0.012649,0.143965
2,高雄市,旗津區,上竹里,"POLYGON ((120.28976 22.57317, 120.28577 22.577...",1391,3254,46.0,63190.09,1374.0,高雄市旗津區上竹里,0.033070,0.033070,3.3%,1345,1374.0,0.097305,1014.897475,0.024816,0.131582
3,高雄市,旗津區,中興里,"POLYGON ((120.30320 22.55568, 120.30284 22.557...",1064,2877,58.0,83960.71,1448.0,高雄市旗津區中興里,0.054511,0.054511,5.5%,1006,1448.0,0.072278,323.622580,0.007911,0.139032
4,屏東縣,潮州鎮,新榮里,"POLYGON ((120.54005 22.54951, 120.54019 22.549...",594,1394,22.0,63775.00,2899.0,屏東縣潮州鎮新榮里,0.037037,0.037037,3.7%,572,2899.0,0.040236,3277.591797,0.080149,0.285110


In [106]:
#gdf_final.to_excel(r'C:\Users\RT009223\滲透率\05_HOLA\result\HOLA2019台灣MP.xlsx',index=0)

# HOLA店點(部分)

In [107]:
HOLA=pd.read_excel('HOLA店點.xlsx')
HOLA

,name,addr,type,lat,lon
0,北屯店,台中市北屯區北屯路407號,0,24.173757,120.697857
1,西屯店,台中市西屯區台灣大道四段528號,0,24.178658,120.624878
2,大墩店,台中市南屯區大墩路533號1F,0,24.149572,120.649650
3,彰化店,彰化縣和美鎮彰美路二段49號,0,24.092321,120.532854
4,Petite大魯閣新時代百貨,台中市東區復興路四段186號6樓,2,24.135776,120.687741
5,HOLA台北內湖店,台北市內湖區新湖三路23號,0,25.062268,121.574347
6,HOLA台北士林店,台北市士林區基河路258號地下1樓,0,25.092449,121.520136
7,Petite台北時代,台北市信義區忠孝東路五段8號6樓,1,25.040626,121.565461
8,HOLA台北中和店,新北市中和區中山路二段291號,0,25.001770,121.495829
9,HOLA台北土城店,新北市土城區青雲路152號三樓,0,24.983359,121.458923


# 畫圖

In [108]:
# 各車站定位
台北車站=[25.047741, 121.517384]
中壢車站=[24.953737, 121.225607]
台中車站=[24.137264, 120.686669]
嘉義車站=[23.479119, 120.441138]

#各縣市邊框顏色
print(gdf_final['縣市'].unique())
site=['基隆市','新北市', '台北市','桃園市','新竹縣','新竹市','苗栗縣','台中市','彰化縣','南投縣',
      '雲林縣','嘉義縣','嘉義市','台南市',
      '高雄市','屏東縣','宜蘭縣','花蓮縣','台東縣']

site_colors={'基隆市':'#F17C67','新北市':'#F17C67', '台北市':'#F17C67',
            '桃園市':'#33A6B8','新竹縣':'#33A6B8','新竹市':'#33A6B8','苗栗縣':'#33A6B8',
            '台中市':'#00896C','彰化縣':'#00896C','南投縣':'#00896C',
            '雲林縣':'#465D4C','嘉義縣':'#465D4C','嘉義市':'#465D4C','台南市':'#465D4C',
            '高雄市':'#6F3381','屏東縣':'#6F3381',
            '宜蘭縣':'#F7D94C','花蓮縣':'#F7D94C','台東縣':'#F7D94C'}

#想畫顏色的變數
variable='ratio2'

#設定色階
high=0.4
low=0.0
p=high/8
print(p)

colorscale = branca.colormap.linear.YlOrRd_09.scale(low, high)
colorscale

gdf_final['id']=gdf_final.index
color=gdf_final.set_index('id')[variable]


['高雄市' '屏東縣' '花蓮縣' '新竹市' '台東縣' '台中市' '嘉義縣' '桃園市' '南投縣' '雲林縣' '新竹縣' '嘉義市'
 '台南市' '基隆市' '苗栗縣' '宜蘭縣' '彰化縣' '新北市' '台北市']
0.05


In [109]:
colorscale

In [110]:
# 顏色 style
def style_function(feature):
    colors = color.get(int(feature['id']), None)
    site_color=site_colors.get(feature['properties']['縣市'])
    return {
        'fillOpacity': 0.5,
        'weight': 1,
        'color':site_color,
        'fillColor': '#fff' if colors is None else colorscale(colors)
    }

In [111]:
# 設定圖例
colormap=colorscale.to_step(index=[0, p*1, p*2, p*3, p*4,p*5,p*6,p*7,p*8])
colormap.caption ='Member Penetration Rates'

In [112]:
mm=folium.Map(location=台北車站,zoom_start=14)
for s in site:
    folium.GeoJson(
        gdf_final[gdf_final['縣市']==s][['geometry','縣市','行政區','村里','戶口','人口','會員數','ratio','percent','MarketPotential','MarketPotential_density','平均消費額','平均消費額_normalization']],
        name=s, 
        style_function=style_function,
        highlight_function=lambda x: {'weight':3, 
                                        'color':site_colors.get(x['properties']['縣市']),
                                        'fillOpacity':0.75},
        smooth_factor=2.0,
        tooltip=folium.features.GeoJsonTooltip(fields=['縣市','行政區','村里','戶口','percent','平均消費額',],
                                                  aliases=['縣市','行政區','村里','戶數','滲透率','平均消費額'], 
                                                  labels=True, 
                                                  sticky=True,
                                                  toLocaleString=True
                                                 )
    ).add_to(mm)

for i in range(HOLA.shape[0]):
    if HOLA.type[i]==0:
        iframe = IFrame(html=heading3(utf2asc(HOLA.name[i])),width=250,height=50)
        popup = folium.Popup(iframe)
        folium.Marker(location=[HOLA.lat[i], HOLA.lon[i]],
                       popup=popup,
                       icon=folium.Icon(icon='info-sign',
                                        color='lightblue')).add_to(mm)
    elif HOLA.type[i]==2:
        iframe = IFrame(html=heading3(utf2asc(HOLA.name[i])),width=250,height=50)
        popup = folium.Popup(iframe)
        folium.Marker(location=[HOLA.lat[i], HOLA.lon[i]],
                       popup=popup,
                       icon=folium.Icon(icon='info-sign',
                                        color='beige')).add_to(mm)
    else:  
        iframe = IFrame(html=heading3(utf2asc(HOLA.name[i])),width=250,height=50)
        popup = folium.Popup(iframe)
        folium.Marker(location=[HOLA.lat[i], HOLA.lon[i]],
                       popup=popup,
                       icon=folium.Icon(icon='info-sign',
                                        color='gray')).add_to(mm)    

colormap.add_to(mm)

folium.LayerControl().add_to(mm)


In [72]:
#mm

In [113]:
#北北基 桃竹苗 中彰投 雲嘉南
mm.save(r'C:\Users\RT009223\TestriteGroup\滲透率專案\05_HOLA\result\台灣2019年HOLA(含Petite)滲透率4.html')